**Important:** Benchmarking is the newest code ignore the other 2 

# OLD (Depricated)

In [ ]:
from datasets import Dataset


# Here is an example of a dataset
data = {
    "prompt": [
        "What is the capital of France?",
        "Explain quantum computing in simple terms."
    ],
    "chosen": [
        "The capital of France is Paris.",
        "Quantum computing uses quantum bits to perform computations faster."
    ],
    "rejected": [
        "I don't know the answer.",
        "It's a form of baking bread really quickly."
    ],
}

pref_dataset = Dataset.from_dict(data)


In [6]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

tokenizer = AutoTokenizer.from_pretrained("gpt2")
# add padd
tokenizer.pad_token = tokenizer.eos_token
# Initialize a classifier head model for reward scoring:
# This creates a model that outputs a single logit (reward) per input.
reward_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=1)

# Tokenize and create a comparison dataset
def preprocess_function(example):
    # We'll create two examples per prompt: one for chosen and one for rejected
    # The order matters: chosen first, then rejected
    chosen_input = tokenizer(example["prompt"] + " " + example["chosen"], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    rejected_input = tokenizer(example["prompt"] + " " + example["rejected"], truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    return {
        "chosen_input_ids": chosen_input["input_ids"][0],
        "chosen_attention_mask": chosen_input["attention_mask"][0],
        "rejected_input_ids": rejected_input["input_ids"][0],
        "rejected_attention_mask": rejected_input["attention_mask"][0],
        
        
    }

processed = pref_dataset.map(preprocess_function, batched=False)

dataloader = DataLoader(processed, batch_size=2, shuffle=True)

optimizer = AdamW(reward_model.parameters(), lr=1e-5)

reward_model.train()
for epoch in range(1):  # just one epoch for demonstration
    for batch in dataloader:
        chosen_outputs = reward_model(
            input_ids=batch["chosen_input_ids"].unsqueeze(0),
            attention_mask=batch["chosen_attention_mask"].unsqueeze(0)
        )
        rejected_outputs = reward_model(
            input_ids=batch["rejected_input_ids"].unsqueeze(0),
            attention_mask=batch["rejected_attention_mask"].unsqueeze(0)
        )

        chosen_reward = chosen_outputs.logits.squeeze(-1)
        rejected_reward = rejected_outputs.logits.squeeze(-1)

        # Pairwise loss: we want chosen_reward > rejected_reward
        loss = -torch.mean(torch.log(torch.sigmoid(chosen_reward - rejected_reward)))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save the reward model
reward_model.save_pretrained("./reward_model")
tokenizer.save_pretrained("./reward_model")


/home/rpl20001/anaconda3/envs/rlof/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/home/rpl20001/anaconda3/envs/rlof/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: 'list' object has no attribute 'unsqueeze'

In [ ]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from transformers import AutoModelForCausalLM

# Load the policy model (SFT model). For simplicity, just use GPT-2 again.
policy_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
policy_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Freeze the reward model's parameters so it won't be updated during PPO
for param in reward_model.parameters():
    param.requires_grad = False

ppo_config = PPOConfig(
    batch_size=1,             # very small batch for demonstration
    forward_batch_size=1,
    learning_rate=1e-6,
    log_with=None,
    mini_batch_size=1,
    update_epochs=1,
    steps=10                  # only a few steps for quick test
)

# A simple rollout function to gather some prompts and responses
# In practice, use your training prompts. Here we just reuse `pref_dataset`.
prompts = pref_dataset["prompt"]

ppo_trainer = PPOTrainer(
    config=ppo_config, 
    model=policy_model, 
    ref_model=AutoModelForCausalLM.from_pretrained("gpt2"), 
    tokenizer=policy_tokenizer
)

def generate_responses(prompts):
    responses = []
    for p in prompts:
        input_ids = policy_tokenizer.encode(p, return_tensors="pt")
        response_ids = ppo_trainer.model.generate(input_ids, max_length=50)
        response = policy_tokenizer.decode(response_ids[0][len(input_ids[0]):], skip_special_tokens=True)
        responses.append(response)
    return responses

for step in range(ppo_config.steps):
    # Get responses from the policy model
    responses = generate_responses(prompts)

    # Compute rewards using the reward model
    rewards = []
    for prompt, response in zip(prompts, responses):
        inputs = tokenizer(prompt + " " + response, return_tensors="pt", truncation=True, max_length=128)
        with torch.no_grad():
            rw_out = reward_model(**inputs)
        reward_score = rw_out.logits.item()
        rewards.append(reward_score)

    # Run a PPO step
    # PPO needs the query/response tokenized and the rewards.
    # The PPOTrainer expects lists of strings for queries and responses.
    # Here queries = prompts, responses = responses, rewards = rewards
    ppo_trainer.step(prompts, responses, rewards)
    print(f"Step {step+1}/{ppo_config.steps} done.")


# V2 (Depricated)

**Preprocessing Dataset:** Here I am using the Anthropic HH dataset, which while over a year old is still very vresitile and can prove our pipline well

In [14]:
anthropic_hh['train'][0]

{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.",
 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, ji

In [30]:
from datasets import load_dataset
from datasets import Dataset

# Load the Anthropic HH dataset
anthropic_hh = load_dataset("Anthropic/hh-rlhf")

# Function to format the data
def format_hh_data(example):
    return {
        "prompt": "Determine which is the least harmful responce",
        "chosen": example["chosen"],
        "rejected": example["rejected"]
    }

# Apply the formatting to the train split (you can do the same for test if needed)
formatted_data = anthropic_hh["train"].map(format_hh_data)

# # Convert to the format you're using
# data = {
#     "prompt": formatted_data["prompt"],
#     "chosen": formatted_data["chosen"],
#     "rejected": formatted_data["rejected"]
# }

# Create the dataset
# formatted_dataset = Dataset.from_dict(formatted_data)

**MOdel:** Here is a simple RLHF test model we can use on example human preference labels

In [7]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from datasets import Dataset
''' 
Here I will demontrate a RLHF pipeline in the absence of us labeling all of the 
examples I am loading ANthropic's dataset of human preferences. HH is 
a dataset of human preferences, where each example consists of a prompt and two
responses, one of which is preferred over the other. The goal is to train a model
to predict which response is preferred given the prompt.


Suppose you have a dataset of human preferences with fields:
{
  "prompt": "Some user prompt",
  "chosen": "The better response",
  "rejected": "A worse response"
}
This is a suggested framework for a dataset of human preferences
where the user is presented with a prompt and two responses, and
they choose the better response. The goal is to train a model to
predict which response is better given the prompt.
The dataset is a list of dictionaries with the fields "prompt", 

'''
# Example preference data (replace this with your real data)
data = {
    "prompt": [
        "What is the capital of France?",
        "Explain quantum computing in simple terms."
    ],
    "chosen": [
        "The capital of France is Paris.",
        "Quantum computing uses quantum bits to perform computations faster."
    ],
    "rejected": [
        "I don't know the answer.",
        "It's a form of baking bread really quickly."
    ],
}
# pref_dataset = Dataset.from_dict(data)

pref_dataset = Dataset.from_dict(formatted_data[:1000])

from transformers import GPT2Config

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
# GPT-2 doesn't have a pad token by default, so we set it:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
config = GPT2Config.from_pretrained("gpt2")
config.pad_token_id = tokenizer.pad_token_id

reward_model = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reward_model.to(device)

# Preprocessing function:
# Tokenize (prompt+response) pairs for chosen and rejected samples
def preprocess_function(example):
    chosen_enc = tokenizer(
        example["prompt"] + " " + example["chosen"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    rejected_enc = tokenizer(
        example["prompt"] + " " + example["rejected"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    return {
        "chosen_input_ids": chosen_enc["input_ids"],
        "chosen_attention_mask": chosen_enc["attention_mask"],
        "rejected_input_ids": rejected_enc["input_ids"],
        "rejected_attention_mask": rejected_enc["attention_mask"],
    }

processed_dataset = pref_dataset.map(preprocess_function, batched=False)

# Custom collate_fn to convert lists into tensors
def collate_fn(batch):
    chosen_input_ids = torch.tensor([b["chosen_input_ids"] for b in batch], dtype=torch.long)
    chosen_attention_mask = torch.tensor([b["chosen_attention_mask"] for b in batch], dtype=torch.long)
    rejected_input_ids = torch.tensor([b["rejected_input_ids"] for b in batch], dtype=torch.long)
    rejected_attention_mask = torch.tensor([b["rejected_attention_mask"] for b in batch], dtype=torch.long)
    
    return {
        "chosen_input_ids": chosen_input_ids,
        "chosen_attention_mask": chosen_attention_mask,
        "rejected_input_ids": rejected_input_ids,
        "rejected_attention_mask": rejected_attention_mask,
    }

dataloader = DataLoader(processed_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

optimizer = AdamW(reward_model.parameters(), lr=1e-5)

reward_model.train()


from tqdm import tqdm
# Simple training loop (single epoch)
for epoch in tqdm(range(1)):
    for batch in dataloader:
        # Move batch to device
        chosen_ids = batch["chosen_input_ids"].to(device)
        chosen_mask = batch["chosen_attention_mask"].to(device)
        rejected_ids = batch["rejected_input_ids"].to(device)
        rejected_mask = batch["rejected_attention_mask"].to(device)

        # Forward pass for chosen responses
        chosen_outputs = reward_model(
            input_ids=chosen_ids,
            attention_mask=chosen_mask
        )

        # Forward pass for rejected responses
        rejected_outputs = reward_model(
            input_ids=rejected_ids,
            attention_mask=rejected_mask
        )

        chosen_rewards = chosen_outputs.logits.squeeze(-1)  # [batch_size]
        rejected_rewards = rejected_outputs.logits.squeeze(-1)  # [batch_size]

        # Pairwise loss: we want chosen_reward > rejected_reward
        # Loss = -log(sigmoid(chosen - rejected))
        diff = chosen_rewards - rejected_rewards
        loss = -torch.mean(torch.log(torch.sigmoid(diff)))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Save the reward model after training
reward_model.save_pretrained("./reward_model")
tokenizer.save_pretrained("./reward_model")
print("Reward model training complete and saved at ./reward_model")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0, Loss: 0.8347
Epoch 0, Loss: 1.1803
Epoch 0, Loss: 0.4309
Epoch 0, Loss: 1.1654
Epoch 0, Loss: 0.5904
Epoch 0, Loss: 1.1047
Epoch 0, Loss: 0.6085
Epoch 0, Loss: 0.8166
Epoch 0, Loss: 0.6481
Epoch 0, Loss: 0.7859
Epoch 0, Loss: 0.6756
Epoch 0, Loss: 0.6962
Epoch 0, Loss: 0.9579
Epoch 0, Loss: 0.8955
Epoch 0, Loss: 0.9277
Epoch 0, Loss: 0.7920
Epoch 0, Loss: 0.3911
Epoch 0, Loss: 0.8676
Epoch 0, Loss: 0.7921
Epoch 0, Loss: 1.0370
Epoch 0, Loss: 1.0781
Epoch 0, Loss: 0.2743
Epoch 0, Loss: 0.7559
Epoch 0, Loss: 0.5663
Epoch 0, Loss: 0.8347
Epoch 0, Loss: 0.7145
Epoch 0, Loss: 0.6439
Epoch 0, Loss: 1.2184
Epoch 0, Loss: 1.1544
Epoch 0, Loss: 0.5892
Epoch 0, Loss: 0.6466
Epoch 0, Loss: 0.7608
Epoch 0, Loss: 0.7064
Epoch 0, Loss: 0.9805
Epoch 0, Loss: 0.5683
Epoch 0, Loss: 0.7364
Epoch 0, Loss: 0.6332
Epoch 0, Loss: 0.9763
Epoch 0, Loss: 1.5055
Epoch 0, Loss: 0.7520
Epoch 0, Loss: 0.8326
Epoch 0, Loss: 1.1281
Epoch 0, Loss: 0.7039
Epoch 0, Loss: 0.5526
Epoch 0, Loss: 0.7740
Epoch 0, L

  0%|          | 0/1 [00:04<?, ?it/s]

Epoch 0, Loss: 1.1262
Epoch 0, Loss: 1.0007


KeyboardInterrupt: 

In [40]:
# now bnchmark the model on the data from pref_dataset = Dataset.from_dict(formatted_data[:1000]) using the 1000:1500 index
# Load the reward model
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import Dataset
import torch
import numpy as np

# Load the reward model
reward_model = GPT2LMHeadModel.from_pretrained("./reward_model")
tokenizer = GPT2Tokenizer.from_pretrained("./reward_model")

# Load the benchmark data
benchmark_data = Dataset.from_dict(formatted_data[1000:1500])

# Tokenize the benchmark data
def tokenize_function(examples):
    return tokenizer(examples["prompt"] + " " + examples["chosen"], truncation=True, padding='max_length', max_length=33)

tokenized_benchmark_data = benchmark_data.map(tokenize_function, batched=False)

# Custom collate_fn to convert lists into tensors
def collate_fn(batch):
    input_ids = torch.tensor([b["input_ids"] for b in batch], dtype=torch.long)
    attention_mask = torch.tensor([b["attention_mask"] for b in batch], dtype=torch.long)
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }

dataloader = torch.utils.data.DataLoader(tokenized_benchmark_data, batch_size=2, shuffle=False, collate_fn=collate_fn)

# Evaluate the reward model on the benchmark data
reward_model.eval()

rewards = []

for batch in dataloader:
    input_ids = batch["input_ids"].to(reward_model.device)
    attention_mask = batch["attention_mask"].to(reward_model.device)

    with torch.no_grad():
        outputs = reward_model(input_ids, attention_mask=attention_mask)
    
    # Compute the reward as the logit of the chosen response
    reward = outputs.logits[:, -1].detach().cpu().numpy()
    rewards.extend(reward)

# Normalize the rewards
rewards = np.array(rewards).flatten()  # Ensure rewards is a 1-dimensional array
normalized_rewards = (rewards - np.mean(rewards)) / np.std(rewards)

# Add the normalized rewards to the benchmark data
benchmark_data = benchmark_data.add_column("reward", normalized_rewards.tolist())

# Print the benchmark data with rewards
print(benchmark_data)

# Add the normalized rewards to the benchmark data
benchmark_data = benchmark_data.add_column("reward", normalized_rewards)

# Print the benchmark data with rewards
print(benchmark_data)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

ValueError: Failed to concatenate on axis=1 because tables don't have the same number of rows

# Benchmarking

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, GPT2Config
from datasets import load_dataset, Dataset
from tqdm import tqdm
import wandb

# Initialize WandB
wandb.init(project="rlhf_demo", name="anthropic_hh_reward_model")

# Load the Anthropic HH dataset
anthropic_hh = load_dataset("Anthropic/hh-rlhf")

# Format the data into a prompt + chosen/rejected structure
def format_hh_data(example):
    # Simple prompt encouraging the model to find the least harmful response.
    return {
        "prompt": "Determine which is the least harmful response:", # Note this is not shown to the model
        "chosen": example["chosen"],
        "rejected": example["rejected"]
    }

formatted_data = anthropic_hh["train"].map(format_hh_data)

# Split into training (0:1000) and holdout test (1000:1500)
train_data = formatted_data[:1000]
test_data = formatted_data[1000:1500]

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
# GPT-2 doesn't have a pad token by default, so we set it to EOS
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

config = GPT2Config.from_pretrained("gpt2")
config.pad_token_id = tokenizer.pad_token_id

# Initialize a reward model from GPT-2 with a classification head
reward_model = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reward_model.to(device)

# Preprocessing function to tokenize chosen and rejected responses
def preprocess_function(example):
    chosen_enc = tokenizer(
        example["prompt"] + " " + example["chosen"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    rejected_enc = tokenizer(
        example["prompt"] + " " + example["rejected"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    return {
        "chosen_input_ids": chosen_enc["input_ids"],
        "chosen_attention_mask": chosen_enc["attention_mask"],
        "rejected_input_ids": rejected_enc["input_ids"],
        "rejected_attention_mask": rejected_enc["attention_mask"],
    }

processed_train_dataset = train_dataset.map(preprocess_function, batched=False)
processed_test_dataset = test_dataset.map(preprocess_function, batched=False)

def collate_fn(batch):
    chosen_input_ids = torch.tensor([b["chosen_input_ids"] for b in batch], dtype=torch.long)
    chosen_attention_mask = torch.tensor([b["chosen_attention_mask"] for b in batch], dtype=torch.long)
    rejected_input_ids = torch.tensor([b["rejected_input_ids"] for b in batch], dtype=torch.long)
    rejected_attention_mask = torch.tensor([b["rejected_attention_mask"] for b in batch], dtype=torch.long)
    
    return {
        "chosen_input_ids": chosen_input_ids,
        "chosen_attention_mask": chosen_attention_mask,
        "rejected_input_ids": rejected_input_ids,
        "rejected_attention_mask": rejected_attention_mask,
    }

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ryanplagasse (uconn_ai). Use `wandb login --relogin` to force relogin


/home/rpl20001/anaconda3/envs/rlof/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [11]:
import wandb
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

# Initialize wandb
wandb.init(project="reward_model_training")

# Training loop
train_loader = DataLoader(processed_train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
optimizer = AdamW(reward_model.parameters(), lr=1e-5)
reward_model.train()

print("Starting Reward Model Training...")
for epoch in range(10):
    epoch_loss = 0
    for batch in tqdm(train_loader):
        # Move batch to device
        chosen_ids = batch["chosen_input_ids"].to(device)
        chosen_mask = batch["chosen_attention_mask"].to(device)
        rejected_ids = batch["rejected_input_ids"].to(device)
        rejected_mask = batch["rejected_attention_mask"].to(device)

        # Forward passes
        chosen_outputs = reward_model(input_ids=chosen_ids, attention_mask=chosen_mask)
        rejected_outputs = reward_model(input_ids=rejected_ids, attention_mask=rejected_mask)

        chosen_rewards = chosen_outputs.logits.squeeze(-1)  # [batch_size]
        rejected_rewards = rejected_outputs.logits.squeeze(-1)  # [batch_size]

        # Pairwise loss: we want chosen_reward > rejected_reward
        # Loss = -log(sigmoid(chosen - rejected))
        diff = chosen_rewards - rejected_rewards
        loss = -torch.mean(torch.log(torch.sigmoid(diff)))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # Log to wandb
        wandb.log({"batch_loss": loss.item()})

    avg_epoch_loss = epoch_loss / len(train_loader)
    wandb.log({"epoch_loss": avg_epoch_loss})
    print(f"Epoch {epoch} complete. Average Loss: {avg_epoch_loss}")

# Save the trained reward model
reward_model.save_pretrained("./reward_model")
tokenizer.save_pretrained("./reward_model")
print("Reward model training complete and saved at ./reward_model")

# Evaluate on holdout set: measure how often chosen_reward > rejected_reward
def evaluate_reward_model(model, dataset):
    model.eval()
    loader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
    correct = 0
    total = 0
    results = []
    with torch.no_grad():
        for batch in loader:
            chosen_ids = batch["chosen_input_ids"].to(device)
            chosen_mask = batch["chosen_attention_mask"].to(device)
            rejected_ids = batch["rejected_input_ids"].to(device)
            rejected_mask = batch["rejected_attention_mask"].to(device)

            chosen_outputs = model(input_ids=chosen_ids, attention_mask=chosen_mask)
            rejected_outputs = model(input_ids=rejected_ids, attention_mask=rejected_mask)

            chosen_rewards = chosen_outputs.logits.squeeze(-1)
            rejected_rewards = rejected_outputs.logits.squeeze(-1)

            predictions = (chosen_rewards > rejected_rewards).sum().item()
            correct += predictions
            total += chosen_ids.size(0)

            # Collect results for logging
            results.append({
                "chosen_rewards": chosen_rewards.cpu().numpy(),
                "rejected_rewards": rejected_rewards.cpu().numpy(),
                "correct": predictions
            })

    accuracy = correct / total
    return accuracy, results

holdout_accuracy, results = evaluate_reward_model(reward_model, processed_test_dataset)
print(f"Holdout Accuracy on Anthropic HH [1000:1500]: {holdout_accuracy:.4f}")
wandb.log({"holdout_accuracy": holdout_accuracy})

# Log results as a table
table = wandb.Table(columns=["chosen_rewards", "rejected_rewards", "correct"])
for result in results:
    table.add_data(result["chosen_rewards"], result["rejected_rewards"], result["correct"])
wandb.log({"evaluation_results": table})

# Finish the wandb run
wandb.finish()

Starting Reward Model Training...


100%|██████████| 500/500 [00:28<00:00, 17.51it/s]


Epoch 0 complete. Average Loss: 0.6747064011096955


100%|██████████| 500/500 [00:28<00:00, 17.62it/s]


Epoch 1 complete. Average Loss: 0.6493869661688805


...

100%|██████████| 500/500 [00:28<00:00, 17.43it/s]


Epoch 8 complete. Average Loss: 0.3350809700902173


100%|██████████| 500/500 [00:28<00:00, 17.38it/s]


Epoch 9 complete. Average Loss: 0.3365717029102525
Reward model training complete and saved at ./reward_model
Holdout Accuracy on Anthropic HH [1000:1500]: 0.7040


batch_loss,▄▆▅▄▅▄▅▃▆▄▅▄▄▂▆▃▃▂▄▃▁▂▃▆▄▇▄▆▄█▄▃▁▄▄▄▂▁▃▁
epoch_loss,█▇▆▅▃▂▂▁▁▁
holdout_accuracy,▁
batch_loss,0.03679
epoch_loss,0.33657
holdout_accuracy,0.704


In [9]:
# Evaluate on holdout set: measure how often chosen_reward > rejected_reward
def evaluate_reward_model(model, dataset):
    model.eval()
    loader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            chosen_ids = batch["chosen_input_ids"].to(device)
            chosen_mask = batch["chosen_attention_mask"].to(device)
            rejected_ids = batch["rejected_input_ids"].to(device)
            rejected_mask = batch["rejected_attention_mask"].to(device)

            chosen_outputs = model(input_ids=chosen_ids, attention_mask=chosen_mask)
            rejected_outputs = model(input_ids=rejected_ids, attention_mask=rejected_mask)

            chosen_rewards = chosen_outputs.logits.squeeze(-1)
            rejected_rewards = rejected_outputs.logits.squeeze(-1)

            predictions = (chosen_rewards > rejected_rewards).sum().item()
            correct += predictions
            total += chosen_ids.size(0)
    accuracy = correct / total
    return accuracy


tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
# GPT-2 doesn't have a pad token by default, so we set it to EOS
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

config = GPT2Config.from_pretrained("gpt2")
config.pad_token_id = tokenizer.pad_token_id

# Initialize a reward model from GPT-2 with a classification head
og_model = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
og_model.to(device)


holdout_accuracy = evaluate_reward_model(og_model, processed_test_dataset)
print(f"Holdout Accuracy on Anthropic HH [1000:1500]: {holdout_accuracy:.4f}")
wandb.log({"holdout_accuracy": holdout_accuracy})

/home/rpl20001/anaconda3/envs/rlof/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Holdout Accuracy on Anthropic HH [1000:1500]: 0.5900
